<a href="https://colab.research.google.com/github/Esbern/Python-for-Planners/blob/main/random_walk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium
import random
import math
from pyproj import Transformer

In [55]:
# Create transformers
wgs_utm = Transformer.from_crs("EPSG:4326", "EPSG:25832", always_xy=True)
utm_wgs = Transformer.from_crs("EPSG:25832", "EPSG:4326", always_xy=True)

# Starting coordinate for Copenhagen in WGS84
start_lat, start_lon = 55.6761, 12.5683

# Transform starting coordinate to EPSG:25832 (UTM)
start_utm_x, start_utm_y = wgs_utm.transform(start_lon, start_lat)

# Create a Folium map centered on the starting location
m = folium.Map(location=[start_lat, start_lon], zoom_start=16)

# Predefined list of colors for each iteration (for random movements)
colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'brown', 'gray', 'black']

# List to store the points in WGS84 (lat, lon, iteration, timestamp)
points = []

# Add the starting point
points.append((start_lat, start_lon, "Start"))
folium.CircleMarker(
    location=[start_lat, start_lon],
    radius=5,
    color='black',
    fill=True,
    fill_color='black',
    popup=f"Start<br>Time: 0"
).add_to(m)

# Initialize UTM coordinates
utm_x = start_utm_x
utm_y = start_utm_y

# Loop 10 times to simulate movement in UTM space
for i in range(10):
    # Generate a random distance in meters (0 to 100)
    distance = random.uniform(0, 100)
    # Generate a random bearing in degrees (0 to 360)
    bearing = random.uniform(0, 360)
    bearing_rad = math.radians(bearing-90) # Comvert from compass bearing

    # Calculate displacement in UTM coordinates
    dx = distance * math.cos(bearing_rad)
    dy = distance * math.sin(bearing_rad)

    # Update UTM coordinates
    utm_x += dx
    utm_y += dy

    # Transform the updated UTM coordinate back to WGS84 (note the order: (lon, lat))
    lon, lat = utm_wgs.transform(utm_x, utm_y)

    points.append((lat, lon, i + 1))

    # Add a circle marker with color based on the iteration
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color=colors[i % len(colors)],
        fill=True,
        fill_color=colors[i % len(colors)],
        popup=f"Iteration {i + 1}"
    ).add_to(m)

# Draw a line connecting the starting point and all subsequent points
if len(points) > 1:
    folium.PolyLine(
        locations=[(pt[0], pt[1]) for pt in points],
        color='blue',
        weight=2.5,
        opacity=0.8
    ).add_to(m)

# Display the map
m
